In [1]:
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
df = pd.read_csv("feature_code.csv", index_col=0)

In [3]:
for r in df.index:
    nums = list()
    end = False
    for i, c in enumerate(df.columns):
        if pd.isnull(df.loc[r,c]):
            end = True
        elif end:
            print('Error: data after end, [{},{}]'.format(r, c))
        elif i%2!=0:
            n = int(df.loc[r,c])
            if not nums:
                nums.append(n)
            elif n == nums[-1] + 1:
                nums.append(n)
            else:
                print('Error: num wrong, [{},{}]'.format(r, c))
print('checked feature code')

checked feature code


In [4]:
feature_code = dict()
for r in df.index:
    feature_code[r] = dict()
    for i, c in enumerate(df.columns):
        if pd.isnull(df.loc[r,c]):
            break
        elif i%2 == 0:
            k = df.loc[r,c]
        else:
            feature_code[r][k] = int(df.loc[r,c])
print(feature_code)

{'萼片数': {}, '萼片颜色': {'绿': 1, '紫红': 2}, '萼片茸毛': {'无': 0, '有': 1}, '完全开放花冠大小1': {}, '完全开放花冠大小2': {}, '花瓣数': {}, '花瓣颜色': {'白': 1, '微绿': 2, '淡红': 3}, '花瓣茸毛': {'无': 0, '有': 1}, '最大花瓣质地': {'薄': 1, '中': 2, '厚': 3}, '子房茸毛': {'无': 0, '有': 1}, '花柱长度': {}, '花柱头开裂数': {}, '柱头裂位': {'浅': 1, '中': 2, '深': 3, '全': 4}, '雌雄蕊相对高度': {'雌低': 1, '等高': 2, '雌高': 3}, '有无花香': {'无': 0, '有': 1}, '果实形状': {'球': 1, '肾': 2, '三角': 3, '四角': 4, '梅花': 5}, '果实大小1（mm）': {}, '果实大小2（mm）': {}, '果皮中部边缘厚度': {}, '种子形状': {'球': 1, '半球': 2, '锥': 3, '似肾': 4, '不规则': 5}, '种子大小1(mm)': {}, '种子大小2(mm)': {}, '百粒重（g）': {}, '种皮颜色': {'棕色': 1, '棕褐色': 2, '褐色': 3}, '叶长cm': {}, '叶宽cm': {}, '叶面积（cm2）': {}, '叶片大小': {'小': 1, '中': 2, '大': 3, '特大': 4}, '叶脉对数': {}, '叶正面颜色': {'黄绿': 1, '淡绿': 2, '绿': 3, '深绿': 4}, '叶正面隆起性': {'平': 1, '稍隆起': 2, '隆起': 3, '强隆起': 4}, '叶形': {'近圆': 1, '卵形': 2, '椭圆': 3, '长椭圆': 4, '披针形': 5}, '叶身形态': {'稍内折': 1, '内折': 2, '平': 3, '稍背卷': 4}, '叶片质地': {'柔软': 1, '中': 2, '硬': 3}, '叶背茸毛': {'无': 0, '有': 1}, '叶齿密度': {'稀': 1, '中': 2, '密': 3}, '叶

In [5]:
def to_num(df):
    for r in df.index:
        if not feature_code[r]:
            df.loc[r] = pd.to_numeric(df.loc[r])
        else:
            print(r, feature_code[r])
            df.loc[r] = df.loc[r].apply(lambda x: x if pd.isnull(x) else feature_code[r][x.strip()])
    return df

In [6]:
muhe = pd.read_csv("tea-muhe.csv", index_col=0)
muhe = to_num(muhe)

最大花瓣质地 {'薄': 1, '中': 2, '厚': 3}
柱头裂位 {'浅': 1, '中': 2, '深': 3, '全': 4}
雌雄蕊相对高度 {'雌低': 1, '等高': 2, '雌高': 3}
着生状态 {'上斜': 1, '稍上斜': 2, '水平': 3, '下垂': 4}
叶正面颜色 {'黄绿': 1, '淡绿': 2, '绿': 3, '深绿': 4}
叶正面隆起性 {'平': 1, '稍隆起': 2, '隆起': 3, '强隆起': 4}
叶形 {'近圆': 1, '卵形': 2, '椭圆': 3, '长椭圆': 4, '披针形': 5}
叶身形态 {'稍内折': 1, '内折': 2, '平': 3, '稍背卷': 4}
叶基形态 {'楔形': 1, '近圆形': 2}
叶缘形态 {'平': 1, '微波': 2, '波': 3}


In [7]:
muhe.loc['完全开放花冠大小1'] = (muhe.loc['完全开放花冠大小1'] + muhe.loc['完全开放花冠大小2'])/2
muhe = muhe.rename({'完全开放花冠大小1':'完全开放花冠大小'})
muhe = muhe.drop(index=['完全开放花冠大小2'])

In [8]:
keys = ['萼片数','完全开放花冠大小','花瓣数','最大花瓣质地','花柱长度','柱头裂位','雌雄蕊相对高度','着生状态','叶长cm','叶宽cm','叶正面颜色','叶正面隆起性','叶形','叶身形态','叶基形态','叶缘形态']
removed_keys = set(muhe.index) - set(keys)
muhe = muhe.drop(removed_keys)

## Correlation

In [9]:
def crowd(df):
    short_rows = list()
    for r in df.index:
        l = list()
        for c in df.columns:
            if not pd.isnull(df.loc[r,c]):
                l.append(df.loc[r,c])
        if len(l) < 40:
            short_rows.append(r)
        l.extend([np.nan for _ in range(df.shape[1]-len(l))])
        df.loc[r] = l
    return df, short_rows

In [10]:
df, short_rows = crowd(muhe)
df = df.drop(short_rows)
print('drop', short_rows)
df.to_csv('results/muhe_num.csv')
df = df.dropna(axis=1)
df = df.T

drop []


In [11]:
drop_cols = list()
for c in df.columns:
    if df[c].std() == 0:
        drop_cols.append(c)
print('drop', drop_cols)
df = df.drop(drop_cols, axis=1)
df

drop []


,萼片数,完全开放花冠大小,花瓣数,最大花瓣质地,花柱长度,柱头裂位,雌雄蕊相对高度,着生状态,叶长cm,叶宽cm,叶正面颜色,叶正面隆起性,叶形,叶身形态,叶基形态,叶缘形态
1,5,3.05,8,2,1.2,2,3,1,8.4,3,3,3,5,4,1,2
2,5,3.75,8,2,1.2,2,3,1,8.7,2.9,1,2,5,3,1,2
3,5,3.7,8,2,1,1,2,2,10.5,3.1,3,2,5,3,1,3
4,5,5.15,6,2,0.8,1,1,1,10.7,3.3,3,2,5,3,1,1
5,5,3.65,8,2,1,1,1,1,7,2.5,3,1,5,2,1,1
6,5,3.5,6,2,1.1,2,2,1,8.3,2.8,3,1,5,2,1,3
7,5,3.55,7,3,0.9,2,1,2,8.8,3.5,3,1,4,1,1,1
8,5,3.05,7,2,1,2,2,2,11.7,4,3,2,4,3,1,1
9,5,4.05,8,2,0.8,2,1,2,9.5,3.6,3,2,4,3,1,1
10,5,3.1,6,3,0.9,2,1,2,9.1,3.8,3,2,4,3,1,2


In [12]:
df_corr = pd.DataFrame()
for r, row in enumerate(df.columns[1:]):
    for c, col in enumerate(df.columns):
        if r >= c:
            rv, pv = stats.pearsonr(df[col], df[row])
            df_corr.loc[row,col] = ('%.3f*' if pv<=0.05 else '%.3f') % rv
# pd.set_option('precision', 3)
display(df_corr)

,萼片数,完全开放花冠大小,花瓣数,最大花瓣质地,花柱长度,柱头裂位,雌雄蕊相对高度,着生状态,叶长cm,叶宽cm,叶正面颜色,叶正面隆起性,叶形,叶身形态,叶基形态
完全开放花冠大小,0.030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
花瓣数,-0.071,0.140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
最大花瓣质地,-0.309,-0.250,0.101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
花柱长度,-0.218,-0.626*,-0.254,0.043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
柱头裂位,-0.019,0.076,0.092,-0.054,-0.382*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
雌雄蕊相对高度,-0.225,-0.408*,-0.089,-0.110,0.774*,-0.378*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
着生状态,-0.100,0.025,-0.045,0.143,-0.064,0.064,-0.126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
叶长cm,-0.031,-0.002,-0.167,0.041,0.094,-0.090,0.060,0.448*,NaN,NaN,NaN,NaN,NaN,NaN,NaN
叶宽cm,-0.207,-0.245,-0.294,0.165,0.279,-0.071,0.109,0.524*,0.803*,NaN,NaN,NaN,NaN,NaN,NaN
叶正面颜色,-0.075,-0.133,-0.222,0.023,-0.002,-0.120,-0.183,0.088,0.004,0.134,NaN,NaN,NaN,NaN,NaN


In [13]:
df_corr.to_csv('results/muhe_corr.csv')

## 方差分析

In [14]:
def avg_std(df):
    avg = df.mean(axis=0)
    std = df.std(axis=0)
    avg_std = pd.concat([avg, std], axis=1, keys=['avg', 'std'])
    avg_std.to_csv('results/muhe_avg_std.csv')
    display(avg_std)

In [15]:
def oneway(df):
    size = len(df.keys())
    mx = np.zeros([size,size], np.float32)
    mx = pd.DataFrame(mx, index=df.keys(), columns=df.keys())
    for k1 in df.keys():
        for k2 in df.keys():
            Fv, pv = stats.f_oneway(df[k1], df[k2])
            mx[k1][k2] = pv
    display(mx)
    display('p <= 0.05:')
    df05 = mx<=0.05
    df05.to_csv('results/muhe_0.05.csv')
    display('p <= 0.01:')
    df01 = mx<=0.01
    df01.to_csv('results/muhe_0.01.csv')

In [16]:
avg_std(df)

,avg,std
萼片数,4.9500,0.220721
完全开放花冠大小,3.6225,0.565453
花瓣数,6.7250,0.905468
最大花瓣质地,2.0250,0.357161
花柱长度,1.2200,0.298887
柱头裂位,1.4500,0.597001
雌雄蕊相对高度,2.2250,0.800240
着生状态,1.7250,0.640012
叶长cm,9.6550,1.813899
叶宽cm,3.6650,0.711643


In [17]:
oneway(df)

,萼片数,完全开放花冠大小,花瓣数,最大花瓣质地,花柱长度,柱头裂位,雌雄蕊相对高度,着生状态,叶长cm,叶宽cm,叶正面颜色,叶正面隆起性,叶形,叶身形态,叶基形态,叶缘形态
萼片数,1.000000e+00,1.092008e-22,1.761569e-19,0.000000e+00,0.000000e+00,0.000000e+00,1.653047e-33,9.809089e-45,8.210551e-27,2.345368e-17,1.054807e-30,8.267661e-44,4.264833e-16,9.966883e-31,0.000000e+00,8.528909e-35
完全开放花冠大小,1.092008e-22,NaN,4.481260e-30,7.132453e-25,1.858154e-37,1.713729e-27,9.858006e-14,4.506956e-23,1.485393e-32,7.682284e-01,6.639237e-08,6.161388e-21,3.758606e-01,1.290242e-08,6.632276e-40,2.191677e-16
花瓣数,1.761569e-19,4.481260e-30,1.000000e+00,4.203895e-45,0.000000e+00,2.802597e-45,3.367923e-37,5.170791e-43,5.761644e-14,1.211281e-27,5.354575e-35,3.825545e-42,5.820866e-27,3.821459e-35,0.000000e+00,3.187941e-38
最大花瓣质地,0.000000e+00,7.132453e-25,4.203895e-45,1.000000e+00,2.108615e-17,1.399504e-06,1.529055e-01,1.148640e-02,2.704716e-40,2.906471e-21,4.667208e-08,3.768711e-01,1.389673e-22,1.077784e-06,4.564242e-22,6.068671e-01
花柱长度,0.000000e+00,1.858154e-37,0.000000e+00,2.108615e-17,1.000000e+00,3.236659e-02,1.130020e-10,2.158824e-05,1.499389e-43,1.728675e-32,3.829218e-21,6.451150e-09,1.272509e-33,2.519414e-19,2.477506e-02,1.930722e-06
柱头裂位,0.000000e+00,1.713729e-27,2.802597e-45,1.399504e-06,3.236659e-02,1.000000e+00,4.911320e-06,5.041119e-02,1.592436e-41,7.871463e-25,6.110400e-14,7.636488e-04,6.863759e-26,1.379513e-12,5.094871e-04,3.083828e-03
雌雄蕊相对高度,1.653047e-33,9.858006e-14,3.367923e-37,1.529055e-01,1.130020e-10,4.911320e-06,1.000000e+00,2.806939e-03,2.178241e-37,9.888939e-13,1.568716e-03,6.392726e-02,9.362536e-14,6.712849e-03,5.835089e-13,1.392827e-01
着生状态,9.809089e-45,4.506956e-23,5.170791e-43,1.148640e-02,2.158824e-05,5.041119e-02,2.806939e-03,1.000000e+00,2.917910e-40,6.845443e-21,7.376150e-10,1.582837e-01,5.591617e-22,1.135281e-08,7.868553e-08,1.836055e-01
叶长cm,8.210551e-27,1.485393e-32,5.761644e-14,2.704716e-40,1.499389e-43,1.592436e-41,2.178241e-37,2.917910e-40,1.000000e+00,1.217747e-31,1.054101e-35,1.299973e-39,2.905175e-31,7.009529e-36,3.643376e-44,3.421616e-38
叶宽cm,2.345368e-17,7.682284e-01,1.211281e-27,2.906471e-21,1.728675e-32,7.871463e-25,9.888939e-13,6.845443e-21,1.217747e-31,1.000000e+00,2.524528e-07,7.798588e-19,5.935763e-01,5.476938e-08,1.366414e-34,2.891500e-15


'p <= 0.05:'

'p <= 0.01:'